In [ ]:
import pandas as pd

In [ ]:
import datetime
from tqdm.notebook import tqdm
import time

# Base URL
base_url = "https://dasway.ess.washington.edu/quakescope/service/classifies/query"

# Start and end dates
start_date = datetime.datetime(2002, 1, 1)
end_date = datetime.datetime(2025, 12, 31)

# Create empty DataFrame to store all results
all_data = pd.DataFrame()

# Loop through each month
current_date = start_date
for _ in tqdm(range((end_date.year - start_date.year) * 12 + end_date.month - start_date.month + 1)):
    # Calculate the end of the current month
    if current_date.month == 12:
        next_month = datetime.datetime(current_date.year + 1, 1, 1)
    else:
        next_month = datetime.datetime(current_date.year, current_date.month + 1, 1)
    
    # Format dates for URL
    start_str = current_date.strftime("%Y-%m-%d")
    end_str = next_month.strftime("%Y-%m-%d")
    
    # Create URL with parameters
    url = f"{base_url}?tid=UW.SHW.&start_time={start_str}&end_time={end_str}&limit=1000"
    try:
        # Get data for current month
        monthly_data = pd.read_csv(url, delimiter="|")
        if monthly_data.columns[0] == "No data found!":
            print(f"No data for {start_str} to {end_str}")
            current_date = next_month
            continue
        
        # Append to all_data
        all_data = pd.concat([all_data, monthly_data], ignore_index=True)
        print(all_data.tail(1))  # Print the last row of the DataFrame
        # Sleep to avoid overwhelming the server
        time.sleep(0.5)
    except Exception as e:
        print(f"Error fetching data for {start_str} to {end_str}: {e}")
    
    # Move to next month
    current_date = next_month
print(f"Total records collected: {len(all_data)}")

  0%|          | 0/288 [00:00<?, ?it/s]

No data for 2002-01-01 to 2002-02-01
No data for 2002-02-01 to 2002-03-01
No data for 2002-03-01 to 2002-04-01
No data for 2002-04-01 to 2002-05-01
No data for 2002-05-01 to 2002-06-01
No data for 2002-06-01 to 2002-07-01
No data for 2002-07-01 to 2002-08-01
No data for 2002-08-01 to 2002-09-01
No data for 2002-09-01 to 2002-10-01
No data for 2002-10-01 to 2002-11-01
No data for 2002-11-01 to 2002-12-01
No data for 2002-12-01 to 2003-01-01
No data for 2003-01-01 to 2003-02-01
No data for 2003-02-01 to 2003-03-01
No data for 2003-03-01 to 2003-04-01
No data for 2003-04-01 to 2003-05-01
No data for 2003-05-01 to 2003-06-01
No data for 2003-06-01 to 2003-07-01
No data for 2003-07-01 to 2003-08-01
No data for 2003-08-01 to 2003-09-01
No data for 2003-09-01 to 2003-10-01
No data for 2003-10-01 to 2003-11-01
No data for 2003-11-01 to 2003-12-01
No data for 2003-12-01 to 2004-01-01
No data for 2004-01-01 to 2004-02-01
No data for 2004-02-01 to 2004-03-01
No data for 2004-03-01 to 2004-04-01
N

In [ ]:
all_data

,trace_id,network_code,station_code,location_code,channel,start_time,eq,px,su
0,UW.SHW.,UW,SHW,NaN,HH,2021-06-21 22:20:28.020,0.998904,3.948020e-12,2.110201e-09
1,UW.SHW.,UW,SHW,NaN,HH,2021-06-21 22:59:08.020,1.000000,9.778896e-17,2.027776e-11
2,UW.SHW.,UW,SHW,NaN,HH,2021-06-21 23:09:08.020,0.007704,9.812896e-01,7.397098e-06
3,UW.SHW.,UW,SHW,NaN,HH,2021-06-21 23:11:28.020,0.999937,7.358728e-09,1.034556e-06
4,UW.SHW.,UW,SHW,NaN,HH,2021-06-21 23:26:48.020,0.000051,9.999485e-01,2.198793e-15
...,...,...,...,...,...,...,...,...,...
26250,UW.SHW.,UW,SHW,NaN,HH,2025-03-30 03:14:40.380,0.996816,3.182885e-03,2.664539e-14
26251,UW.SHW.,UW,SHW,NaN,HH,2025-03-30 03:15:40.380,0.166588,8.333101e-01,3.475742e-08
26252,UW.SHW.,UW,SHW,NaN,HH,2025-03-30 05:34:00.380,0.078771,9.189416e-01,8.604250e-09
26253,UW.SHW.,UW,SHW,NaN,HH,2025-03-30 05:35:40.380,0.023710,8.151136e-01,7.638545e-07


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Convert start_time to datetime if not already
all_data['start_time'] = pd.to_datetime(all_data['start_time'])

# Create a month column for grouping
all_data['month'] = all_data['start_time'].dt.to_period('M')

# Filter rows where su > 0.7
filtered_data = all_data[all_data['su'] > 0.7]

# Group by month and count
monthly_counts = filtered_data.groupby('month').size()

# Convert the period index to datetime for better plotting
monthly_counts.index = monthly_counts.index.to_timestamp()

# Plot as a bar chart
plt.figure(figsize=(14, 7))
ax = plt.subplot(111)

# Create bar chart with custom color and edge color
bars = ax.bar(monthly_counts.index, monthly_counts.values, 
              width=25, color='royalblue', edgecolor='black', alpha=0.8)

# Format x-axis to show dates properly
# ax.xaxis.set_major_locator(mdates.YearLocator())
# ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
# ax.xaxis.set_minor_locator(mdates.MonthLocator())

# # Add title and labels with improved styling
# plt.title('Monthly Earthquake Events with SU > 0.7', fontsize=16, fontweight='bold')
# plt.xlabel('Date', fontsize=12)
# plt.ylabel('Number of Events', fontsize=12)

# # Improve overall styling
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.xticks(rotation=45)
# plt.tight_layout()

# # Add total count as text
# total_events = monthly_counts.sum()
# plt.text(0.02, 0.95, f'Total Events: {total_events}', 
#          transform=ax.transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

plt.show()